In [1]:
from keras.models import Sequential
from keras import metrics
from keras.layers import *
from keras.optimizers import *
from keras import callbacks

import pandas as pd
import numpy as np

import collections


Using TensorFlow backend.


In [2]:
df_raw = pd.read_csv('./call_no3_csv.csv',encoding='utf-8',header=0)

In [3]:
print(df_raw[:5])
sample_n = len(df_raw)

     features CLASS_NO1
0       海洋与海岸  LIGONGKE
1   头痛151个怎么办  LIGONGKE
2       丘陵与平原  LIGONGKE
3  颈椎病132个怎么办  LIGONGKE
4       山地与高原  LIGONGKE


In [4]:
# Drop noisy characters
def drop_noise(x):
    noise = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '，', '：', '。', '“', '”', '１','２','３','４','５','６','７','８','９','０', ':', ' ', '的']
    result = x.copy()
    for i, ii in enumerate(x):
        for j, jj in enumerate(noise):
            if jj == ii:
                result.remove(ii)
                
    return result

In [5]:
listizer = lambda x: list(x)
setizer = lambda x: set(x)

df_drop_noise=df_raw.copy()

df_drop_noise['features'] = df_drop_noise['features'].apply(listizer)
df_drop_noise['features'] = df_drop_noise['features'].apply(drop_noise)

df_str=df_drop_noise.copy()
#df_set=df_drop_noise.copy()
df_list=df_drop_noise.copy()

df_str['features'] = df_str['features'].apply(listizer)

#df_set['features'] = df_set['features'].apply(setizer)
df_list['features'] = df_list['features'].apply(listizer)

In [6]:
df_list[:5]

,features,CLASS_NO1
0,"[海, 洋, 与, 海, 岸]",LIGONGKE
1,"[头, 痛, 个, 怎, 么, 办]",LIGONGKE
2,"[丘, 陵, 与, 平, 原]",LIGONGKE
3,"[颈, 椎, 病, 个, 怎, 么, 办]",LIGONGKE
4,"[山, 地, 与, 高, 原]",LIGONGKE


Bag of words

In [7]:
# Create Corpus
vocab = []
for words in df_str['features']:
    for word in words:
        vocab.append(word)
vocab_list = vocab
vocab = set(vocab)
#print(vocab)
freq_dict = collections.Counter(vocab_list)

# Create dict
word2index = {}
ii = 1
for i,word in enumerate(vocab):
    word2index[word] = ii
    ii += 1

print(len(word2index))

4487


In [8]:
# Frequency

freq_dict = collections.Counter(vocab_list)
print(freq_dict.most_common(50))

print(len(word2index))

[('国', 12641), ('中', 10596), ('文', 8114), ('学', 7149), ('与', 6941), ('研', 5438), ('究', 5271), ('人', 4390), ('大', 4217), ('集', 4166), ('史', 4129), ('代', 4037), ('民', 3741), ('年', 3433), ('生', 3310), ('法', 3060), ('理', 2989), ('化', 2968), ('论', 2851), ('一', 2848), ('全', 2797), ('编', 2783), ('书', 2697), ('家', 2682), ('事', 2633), ('新', 2616), ('小', 2406), ('经', 2377), ('本', 2364), ('世', 2352), ('物', 2174), ('古', 2155), ('会', 2096), ('地', 2088), ('之', 1976), ('发', 1967), ('教', 1925), ('传', 1911), ('战', 1892), ('业', 1854), ('我', 1837), ('子', 1827), ('和', 1823), ('术', 1815), ('图', 1790), ('四', 1782), ('美', 1768), ('界', 1738), ('记', 1725), ('实', 1703)]
4487


In [9]:
# Least Common Func
from operator import itemgetter
import heapq
import collections
def least_common_values(array, to_find=None):
    counter = collections.Counter(array)
    if to_find is None:
        return sorted(counter.items(), key=itemgetter(1), reverse=False)
    return heapq.nsmallest(to_find, counter.items(), key=itemgetter(1))

In [10]:
least_common_values(freq_dict, 9)

[('疡', 1),
 ('堵', 1),
 ('佯', 1),
 ('槛', 1),
 ('茆', 1),
 ('給', 1),
 ('蜷', 1),
 ('─', 1),
 ('勐', 1)]

In [11]:
index2word = {ii: word for word, ii in word2index.items()}

In [12]:
word2index["波"]

2149

In [13]:


# Convert using bag of words
vocab_size = len(word2index)
#features = pd.DataFrame(np.random.uniform(0,1,size=(sample_n, 100)))
#features = np.zeros(shape=(sample_n, vocab_size))
max_len = 80
features = np.zeros(shape=(sample_n, max_len), dtype=np.int32)

In [14]:
for ii, row in enumerate(df_list['features']):
    #print(row)
    for jj, word in enumerate(row):
        if len(row) <= 80:
            len_row = 80
        else:
            len_row = len(row)
        
        try:
            features[ii][len_row - jj] = word2index[word]
        except:
            pass
    if ii % 20000 == 0:
        print(ii/sample_n*100)
        print(features.shape)

0.0
(66431, 80)
30.106426216675953
(66431, 80)
60.212852433351905
(66431, 80)
90.31927865002784
(66431, 80)


In [15]:
features[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0, 1696, 1361,
       1054, 1003, 4247])

ONE HOT ENCODE LABEL

In [17]:
df_label=df_list.copy()
def upperiser(x):
    if str(x) != 'nan':
        return str(x).upper()
    else:
        return 'WENKE'

df_label['CLASS_NO1'] = df_label['CLASS_NO1'].apply(upperiser)

In [18]:
df_label=df_label.drop('features', axis=1)
df_label[:5]

,CLASS_NO1
0,LIGONGKE
1,LIGONGKE
2,LIGONGKE
3,LIGONGKE
4,LIGONGKE


In [19]:
np_label = np.array(df_label['CLASS_NO1'])

set_label = set(np_label)
set_label

{'LIGONGKE', 'WENKE'}

In [20]:
num2class = {'1': 'WENKE',
'2': 'LIGONGKE'}

In [21]:
class2num = {ii: word for word, ii in num2class.items()}
class2num

{'LIGONGKE': '2', 'WENKE': '1'}

In [22]:
def class2ohfunc(classno):
    oh = [0,0]
    oh[int(class2num[classno])-1] = 1
    return np.array(oh)

In [23]:
df_label['CLASS_NO1'] = df_label['CLASS_NO1'].apply(class2ohfunc)
df_label[:3]

,CLASS_NO1
0,"[0, 1]"
1,"[0, 1]"
2,"[0, 1]"


testing set

In [27]:
features.shape
print(vocab_size)
print(df_list[-2421:3205].shape)

4487
(0, 2)


In [28]:
labels = np.array(df_label['CLASS_NO1'])
labels = np.array(labels.tolist())
trainX = features[:-6124]
trainY = labels[:-6124]
testX = features[-6124:]
testY = labels[-6124:]

In [29]:
print(testY[0:2])

[[1 0]
 [1 0]]


In [30]:
#trainX = trainX.astype('float32')
#trainY = trainY.astype('float32')
trainY = np.reshape(trainY, newshape=(len(trainX),2))
#testX = testX.astype('float32')
#testY = testY.astype('float32')
testY = np.reshape(testY, newshape=(len(testX),2))

In [31]:
trainX.shape, trainY.shape

((60307, 80), (60307, 2))

Scikit-Learn Test

In [34]:
from sklearn import datasets

ImportError: No module named 'sklearn'

In [35]:
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target

NameError: name 'datasets' is not defined

In [35]:
Y[:5]

NameError: name 'Y' is not defined

In [36]:
trainYs = np.zeros((len(trainX)))
for i, ii in enumerate(trainY):
    trainYs[i] = ii[0]

testYs = np.zeros((len(testX)))
for i, ii in enumerate(testY):
    testYs[i] = ii[0]

In [37]:
trainYs[:5]

array([ 0.,  0.,  0.,  0.,  0.])

In [38]:
logreg = linear_model.LogisticRegression()
logreg.fit(trainX, trainYs)

NameError: name 'linear_model' is not defined

In [39]:
logreg.score(trainX, trainYs)

NameError: name 'logreg' is not defined

In [34]:
logreg.score(testX, testYs)

0.92553886348791636

TRAINING SOON STARTS

In [35]:
#T

#### vocab_size = len(word2index)
print(features.shape)
print(vocab_size)

Build the Nerual Network with TFLearn's sample code

In [34]:
model = Sequential()

model.add(Embedding(vocab_size, 800, input_length=80))

model.add(LSTM(1200))
model.add(Dropout(0.5))

model.add(Dense(600, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(80, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))

adam_op = Adam(lr=0.001)
#adam_op = Adam()

model.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=[metrics.categorical_accuracy])

In [35]:
model.fit(trainX, trainY, batch_size=32, epochs=6, verbose=1, callbacks=[cbk.TensorBoard(log_dir='./Graph', histogram_freq=22, write_graph=True, write_grads=True, write_images=True)])

Epoch 1/6
60307/60307 [==============================] - 615s - loss: 0.2839 - categorical_accuracy: 0.8959   
Epoch 2/6
60307/60307 [==============================] - 615s - loss: 0.1992 - categorical_accuracy: 0.9264   
Epoch 3/6
60307/60307 [==============================] - 613s - loss: 0.1715 - categorical_accuracy: 0.9362   
Epoch 4/6
60307/60307 [==============================] - 612s - loss: 0.1519 - categorical_accuracy: 0.9438   
Epoch 5/6
60307/60307 [==============================] - 611s - loss: 0.1372 - categorical_accuracy: 0.9489   
Epoch 6/6
60307/60307 [==============================] - 613s - loss: 0.1228 - categorical_accuracy: 0.9540   


In [45]:
#keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
from keras import callbacks
callbacks.TensorBoard(log_dir='./keraslogs')

In [37]:
score = model.evaluate(trainX, trainY)
print("")
print(score[1])

# 5 epochs: 0.98365032251645745

60307/60307 [==============================] - 158s   

0.962806970998


In [38]:
score = model.evaluate(testX, testY)
print("")
print(score[1])

# 5 epochs: 0.9301110387315632
# another: 0.933050293926

6124/6124 [==============================] - 16s    

0.928314826911


In [31]:
try01 = testX[97]
result = model.predict(testX)
print(result[97])

[ 1.  0.]


In [32]:
# Prediction starts here

#pred_list = np.zeros(shape=(len(testX),1))
#for i, test_feature in enumerate(testX):
#    result = model.predict([test_feature])
 #   pred_list[i][0] = result[0][0]
    
pred_list = np.zeros(shape=(len(testX),2))
results = model.predict(testX)
for i, res in enumerate(results):
    result = res
    if i == 1:
        print(res)
    for jj in range(2):
        pred_list[i][jj] = result[jj]

[ 1.  0.]


In [40]:
pred_list[:100]

array([[  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  7.18884349e-01,   2.81115621e-01],
       [  

In [34]:
def toClassNo(npArray):
    i = 0
    val = npArray[0]
    for jj in range(len(npArray)-1):
        if npArray[jj+1] > val:
            val = npArray[jj+1]
            i = jj + 1
    return num2class[str(i+1)]

In [35]:
pred_result_list = [None] * len(testX)
for i, result in enumerate(pred_list):
    pred_result_list[i] = toClassNo(result)

print(pred_result_list[:5])
len(pred_result_list)


['WENKE', 'WENKE', 'WENKE', 'WENKE', 'WENKE']


6124

In [36]:
df_raw[-6124:-6124+5]


,features,CLASS_NO1
60307,柏杨全集,WENKE
60308,四部叢刊 :四編,WENKE
60309,四部叢刊 :四編,WENKE
60310,四部叢刊 :四編,WENKE
60311,四部叢刊 :四編,WENKE


In [37]:
df_pred = pd.DataFrame(pred_result_list)
tempdf = df_raw.copy()
tempdf = tempdf.drop(tempdf.index[:-6124])
tempdfa = tempdf['features']
tempdfb = tempdf['CLASS_NO1']
output = pd.concat([tempdfa, tempdfb], axis=1)

In [38]:
df_pred.to_csv('output1.csv')
output.to_csv('output2.csv')